In [3]:
import requests
import json
import pandas as pd
import numpy as np

from datetime import datetime,timedelta

In [4]:
import dotenv
import os

# Load secret file
dotenv.load_dotenv('.env')

headers = {
  "Authorization":f"apikey {os.environ.get('apikey')}"
}
url_carparks = 'https://api.transport.nsw.gov.au/v1/carpark'
# url_carpark_history = 'https://api.transport.nsw.gov.au/v1/carpark/history'

list_of_carparks = requests.get(url_carparks, headers=headers).json()

# Create a json file and write the data if it doesn't exist
if not os.path.exists('./data/carparks.json'):
  with open('./data/carparks.json', 'w') as f:
    json.dumps(list_of_carparks, f)

# The format of the file itself is a dictionary, thus requiring some creativity to read
with open('./data/carparks.json') as f:
  data = json.load(f)

# Convert the dictionary to a dataframe
df_carparks = pd.DataFrame.from_dict(data, orient='index')
# Resetting the index to label the columns afterwards
df_carparks = df_carparks.reset_index()
df_carparks.columns = ['API_ID', 'CarParkName']

df_carparks

,API_ID,CarParkName
0,1,Tallawong Station Car Park (historical only)
1,10,Warriewood Car Park
2,11,Narrabeen Car Park
3,12,Mona Vale Car Park
4,13,Dee Why Car Park
5,14,West Ryde Car Park
6,15,Sutherland East Parade Car Park
7,16,Leppington Car Park
8,17,Edmondson Park South Car Park
9,18,St Marys Car Park


Attempting to create a date_time function that gives the dates of all days in an array from 31st December 2023 to a target date that is separated by a time delta

In [6]:
def date_getter(td):
  # Array that stores the dates to be searched for
  date_period_array = []
  
  # The last date to be searched for
  cutoff_date = datetime(2023,12,31)
  target_date = cutoff_date - td
  
  # Ensure that records of each day are obtained
  delta = timedelta(days=1)
  
  while target_date <= cutoff_date:
    date_period_array.append(target_date.strftime("%Y-%m-%d"))
    target_date += delta

  return date_period_array

In [9]:
import os

def carpark_history_period(facility, dates_array):
    data_array = []
    json_file_path = f"./data/carpark_history_facility_{facility}.json"
    
    # Request header
    headers = {
        "Authorization":f"apikey {os.environ.get('apikey')}"
    }
    
    # Deleting file if it exists
    if os.path.exists(json_file_path):
        os.remove(json_file_path)
    
    # Make a request for each date
    for date in dates_array:
        url = f'https://api.transport.nsw.gov.au/v1/carpark/history?facility={facility}&eventdate={date}'
    
        response = requests.get(url, headers=headers).json()
    
        if data_array == []:
            data_array = response
        else:
            data_array = data_array + response
    
    # Saving data to json file
    with open(json_file_path, 'w') as f:
        json.dump(data_array, f)

    # Reading json file
    with open(json_file_path) as f:
        data = json.load(f)
    
    # Converting read data into a pandas dataframe
    return pd.DataFrame(data)

In [10]:
df = carpark_history_period(facility=14, dates_array=date_getter(td=timedelta(days=2)))

df.head()

,tsn,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id
0,211420,757083975,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '215031', 'total': '0', 'monthlies': ...",2023-12-29T00:06:15,14,West Ryde Car Park,211420TPR001
1,211420,757084211,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '215032', 'total': '1', 'monthlies': ...",2023-12-29T00:10:11,14,West Ryde Car Park,211420TPR001
2,211420,757084439,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '215033', 'total': '0', 'monthlies': ...",2023-12-29T00:13:59,14,West Ryde Car Park,211420TPR001
3,211420,757085039,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '215033', 'total': '0', 'monthlies': ...",2023-12-29T00:23:59,14,West Ryde Car Park,211420TPR001
4,211420,757085641,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '215033', 'total': '0', 'monthlies': ...",2023-12-29T00:34:01,14,West Ryde Car Park,211420TPR001


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945 entries, 0 to 944
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   tsn                945 non-null    object
 1   time               945 non-null    object
 2   spots              945 non-null    object
 3   zones              945 non-null    object
 4   ParkID             945 non-null    object
 5   occupancy          945 non-null    object
 6   MessageDate        945 non-null    object
 7   facility_id        945 non-null    object
 8   facility_name      945 non-null    object
 9   tfnsw_facility_id  945 non-null    object
dtypes: object(10)
memory usage: 74.0+ KB
